## This is the simulation of the circuit on page 12

In [22]:
import sys
import os

sys.path.append(os.path.abspath(".."))  

In [23]:
import cirq
from qudit_cirq.circuit_builder import create_circuit
from qudit_cirq.utils import format_out, printVector, tensor_product
from qudit_cirq.qudit import quditXGate, quditZGate, quditHGate, quditCNOTGate, state_vector, qudit_measure

#### Description of the algorithm:



- Prepare the Generalized GHZ State (ψ₀): This is a multi-qudit entangled state, initialized for n clients.
- Teleportation Using a Generalized Bell State (|β⟩): The server teleports each qudit in ψ₀ to the clients.
- Client Actions:
Each client runs an LDP (Local Differential Privacy) Algorithm.
Applies a generalized Z gate based on the output of the LDP Algorithm.
Applies a generalized Hadamard (H) gate.
- Measurement: Each client measures their qudit in the computational basis.
Server Aggregation: The server collects the measurement results, computes a de-biased sum, and applies the formula provided.



In [50]:
import cirq
import numpy as np
from typing import List

In [ ]:
d = 3  # Dimension for qudits
n = 3  # Number of clients
kappa = 3  # Parameter of Algorithm 1
gamma = 0.5  # Probability parameter for the LDP algorithm

In [52]:
def prepare_ghz_state(circuit, qudits, d):
    # Apply generalized Hadamard (using quditHGate) on the first qudit
    circuit.append(quditHGate(d).on(qudits[0]))
    
    # Entangle the qudits to create a GHZ state
    for i in range(1, len(qudits)):
        circuit.append(quditCNOTGate(d).on(qudits[i - 1], qudits[i]))  # Apply generalized CNOT


In [53]:
def ldp_algorithm(x, kappa, gamma):
    if np.random.rand() < gamma:
        return np.random.randint(0, kappa) 
    else:
        return x

In [54]:
qudits = [cirq.LineQid(i, dimension=d) for i in range(n)]
circuit = cirq.Circuit()

In [55]:
prepare_ghz_state(circuit, qudits, d)

In [56]:
class quditZGate(cirq.Gate):
    def __init__(self, d, exponent=1):
        super(quditZGate, self).__init__()
        self.d = d
        self.exponent = exponent

    def _qid_shape_(self):
        return (self.d,)

    def _unitary_(self):
        z_matrix = np.diag([np.exp(2j * np.pi * j * self.exponent / self.d) for j in range(self.d)])
        return z_matrix

    def _circuit_diagram_info_(self, args):
        return f"Z(d={self.d})^{self.exponent}"

    def with_exponent(self, exponent):
        return quditZGate(self.d, exponent)

In [57]:
for i in range(n):
    x_i = np.random.randint(0, d) 
    y_i = ldp_algorithm(x_i, kappa, gamma)
    
    z_gate_yi = quditZGate(d).with_exponent(y_i)
    circuit.append(z_gate_yi.on(qudits[i]))
    
    circuit.append(quditHGate(d).on(qudits[i]))
    
    circuit.append(qudit_measure(qudits[i], key=f'm_qudit{i}'))

In [58]:
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)

In [59]:
z_values = [result.measurements[f'm_qudit{i}'][0][0] for i in range(n)]
z_sum = sum(z_values) % d
z_debiased = (z_sum - gamma * (kappa - 1) * n / 2) / (1 - gamma)

In [60]:
print("Circuit:")
print(circuit)

Circuit:
0 (d=3): ───H(d=3)───C───Z(d=3)^1───H(d=3)─────M('m_qudit0')───────────────────
                     │
1 (d=3): ────────────X───C──────────Z(d=3)^1───H(d=3)──────────M('m_qudit1')───
                         │
2 (d=3): ────────────────X──────────Z(d=3)^2───H(d=3)──────────M('m_qudit2')───


In [61]:
print("\nMeasurement results (z values):", z_values)
print("De-biased sum:", z_debiased)


Measurement results (z values): [1, 0, 0]
De-biased sum: -1.0
